In [ ]:
#!/usr/bin/env python3

"""
Autonomous Goal Scoring Robot - Complete Workflow
Receives position data from laptop vision system via HiveMQ
Receives goal status from goal detection camera
Sends servo commands to ESP32
Units: CENTIMETERS for position, RADIANS for angles
Uses iRobot Create 3 Drive Goals API (DriveDistance and RotateAngle)
"""

import rclpy
from rclpy.node import Node
from rclpy.action import ActionClient
from geometry_msgs.msg import PoseStamped
from irobot_create_msgs.action import DriveDistance, RotateAngle, Dock
from std_msgs.msg import Bool
import paho.mqtt.client as mqtt
import ssl
import json
import math
import time


class AutonomousGoalScorer(Node):
    def __init__(self):
        super().__init__('autonomous_goal_scorer')
        
        # ===== CONFIGURATION - EDIT THESE =====
        # HiveMQ Cloud MQTT Broker
        self.MQTT_BROKER = "71b19996472b44ef8901c930925513fd.s1.eu.hivemq.cloud"
        self.MQTT_PORT = 8883
        self.MQTT_USERNAME = "hiveular"
        self.MQTT_PASSWORD = "1HiveMind"
        self.MQTT_TOPIC_POSITION = "robot/position"
        self.MQTT_TOPIC_SERVO = "servo"
        self.MQTT_TOPIC_GOAL_STATUS = "ball/goal_status"
        
        # Shooting position (in centimeters, theta in radians)
        self.SHOOTING_POSITION_CM = {
            'x': 110.0,
            'y': 80.0,
            'theta': 0.0
        }
        
        # Behavior parameters (in centimeters)
        self.APPROACH_DISTANCE_CM = 30.0  # 30cm from ball
        self.ENTRY_DRIVE_CM = 40.0  # Drive 40cm to enter april tag region
        self.BALL_REST_TIME = 3.0  # seconds ball must be stationary
        self.POSITION_TOLERANCE_CM = 5.0  # ±5cm position tolerance
        self.ANGLE_TOLERANCE_RAD = 0.174  # ±10 degrees
        self.CLASP_WAIT_TIME = 3.0  # Wait time for clasp action
        self.CELEBRATION_TIME = 10.0  # Celebration duration
        # ======================================
        
        # State tracking
        self.state = 'WAITING_OUTSIDE'
        self.vision_data = None
        self.goal_status = 'WAITING'
        self.ball_last_position = None
        self.ball_stationary_start = None
        self.retry_count = 0
        self.MAX_RETRIES = 3
        self.data_received = False
        
        # ROS 2 Action clients
        self.drive_client = ActionClient(self, DriveDistance, '/drive_distance')
        self.rotate_client = ActionClient(self, RotateAngle, '/rotate_angle')
        
        self.get_logger().info('Waiting for action servers...')
        self.drive_client.wait_for_server()
        self.rotate_client.wait_for_server()
        self.get_logger().info('✓ Action servers connected!')
        
        # MQTT setup with TLS for HiveMQ Cloud
        self.mqtt_client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
        self.mqtt_client.username_pw_set(self.MQTT_USERNAME, self.MQTT_PASSWORD)
        self.mqtt_client.tls_set(cert_reqs=ssl.CERT_REQUIRED, tls_version=ssl.PROTOCOL_TLS)
        self.mqtt_client.on_connect = self.on_mqtt_connect
        self.mqtt_client.on_message = self.on_mqtt_message
        
        try:
            self.get_logger().info(f'Connecting to HiveMQ Cloud at {self.MQTT_BROKER}...')
            self.mqtt_client.connect(self.MQTT_BROKER, self.MQTT_PORT, 60)
            self.mqtt_client.loop_start()
            self.get_logger().info('MQTT connection initiated')
        except Exception as e:
            self.get_logger().error(f'Failed to connect to MQTT broker: {e}')
        
        # Timer to check ball position periodically
        self.create_timer(0.1, self.state_machine)
        
        self.get_logger().info('=== Autonomous Goal Scorer Ready ===')
        self.get_logger().info('Units: Positions in CM, Angles in RADIANS')
        self.get_logger().info('Waiting outside april tag region for ball...')
        self.get_logger().info('Press Ctrl+C to stop at any time')
    
    def on_mqtt_connect(self, client, userdata, flags, reason_code, properties):
        """MQTT connection callback"""
        if reason_code == 0:
            self.get_logger().info('✓ Connected to HiveMQ Cloud MQTT broker')
            client.subscribe(self.MQTT_TOPIC_POSITION)
            client.subscribe(self.MQTT_TOPIC_GOAL_STATUS)
            self.get_logger().info(f'✓ Subscribed to topics: {self.MQTT_TOPIC_POSITION}, {self.MQTT_TOPIC_GOAL_STATUS}')
        else:
            self.get_logger().error(f'MQTT connection failed with code: {reason_code}')
    
    def on_mqtt_message(self, client, userdata, msg):
        """MQTT message callback"""
        try:
            if msg.topic == self.MQTT_TOPIC_POSITION:
                data = json.loads(msg.payload.decode())
                self.vision_data = data
                
                if not self.data_received:
                    self.data_received = True
                    self.get_logger().info('✓ Receiving vision data from laptop!')
                    if data.get('ball_found'):
                        self.get_logger().info(f"  Ball: ({data['ball_x']:.1f}, {data['ball_y']:.1f}) cm")
            
            elif msg.topic == self.MQTT_TOPIC_GOAL_STATUS:
                # Format: "GOAL|timestamp" or "MISS|timestamp" or "WAITING|timestamp"
                payload = msg.payload.decode()
                status = payload.split('|')[0]
                self.goal_status = status
                
        except json.JSONDecodeError:
            self.get_logger().error(f'Invalid JSON from MQTT: {msg.payload}')
        except Exception as e:
            self.get_logger().error(f'Error processing MQTT message: {e}')
    
    def publish_servo_command(self, command):
        """Publish command to servo topic"""
        try:
            #message = json.dumps(command)
            message = command
            print('Sending ', command)
            self.mqtt_client.publish(self.MQTT_TOPIC_SERVO, message, qos=0)
            self.get_logger().info(f'📡 Servo command sent: {command}')
        except Exception as e:
            self.get_logger().error(f'Failed to publish servo command: {e}')
    
    def state_machine(self):
        """Main state machine for robot behavior"""
        
        if self.state == 'WAITING_OUTSIDE':
            self.check_ball_stationary()
        
        # Other states are handled by action callbacks
        # No polling needed for active movement states
    
    def check_ball_stationary(self):
        """Check if ball has entered region and is stationary"""
        if self.vision_data is None or not self.vision_data.get('ball_found'):
            return
        
        ball_pos = {
            'x': self.vision_data['ball_x'],
            'y': self.vision_data['ball_y']
        }
        
        # Check if ball position has changed
        if self.ball_last_position is None:
            self.ball_last_position = ball_pos.copy()
            self.ball_stationary_start = time.time()
            return
        
        # Calculate distance moved (in cm)
        dx = ball_pos['x'] - self.ball_last_position['x']
        dy = ball_pos['y'] - self.ball_last_position['y']
        distance_moved = math.sqrt(dx**2 + dy**2)
        
        if distance_moved < 1.0:  # Ball hasn't moved much (1cm threshold)
            if time.time() - self.ball_stationary_start >= self.BALL_REST_TIME:
                self.get_logger().info('🎯 Ball stationary for 3+ seconds! Starting sequence...')
                self.get_logger().info(f'    Ball at: ({ball_pos["x"]:.1f}, {ball_pos["y"]:.1f}) cm')
                self.start_entry_drive()
        else:
            # Ball moved, reset timer
            self.ball_last_position = ball_pos.copy()
            self.ball_stationary_start = time.time()
    
    def start_entry_drive(self):
        """Drive 40cm forward to enter april tag region"""
        self.get_logger().info('=' * 60)
        self.get_logger().info('🚀 ENTERING APRIL TAG REGION')
        self.get_logger().info('=' * 60)
        
        self.state = 'ENTERING_REGION'
        
        goal_msg = DriveDistance.Goal()
        goal_msg.distance = self.ENTRY_DRIVE_CM / 100.0  # Convert to meters
        goal_msg.max_translation_speed = 0.3
        
        self.get_logger().info(f'Driving {self.ENTRY_DRIVE_CM} cm forward...')
        send_goal_future = self.drive_client.send_goal_async(goal_msg)
        send_goal_future.add_done_callback(self.entry_drive_response)
    
    def entry_drive_response(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().error('❌ Entry drive REJECTED!')
            self.state = 'WAITING_OUTSIDE'
            return
        
        self.get_logger().info('✓ Entry drive accepted')
        result_future = goal_handle.get_result_async()
        result_future.add_done_callback(self.entry_drive_complete)
    
    def entry_drive_complete(self, future):
        """After entering region, start approaching ball"""
        self.get_logger().info('✓ Entered april tag region!')
        time.sleep(1.0)  # Allow vision to update
        
        self.state = 'ROTATING_TO_BALL'
        self.retry_count = 0
        self.rotate_to_ball()
    
    def rotate_to_ball(self):
        """Calculate and execute rotation to face the ball"""
        if not self.vision_data or not self.vision_data.get('ball_found') or not self.vision_data.get('robot_found'):
            self.get_logger().error('No robot or ball position available!')
            self.state = 'WAITING_OUTSIDE'
            return
        
        ball_x = self.vision_data['ball_x']
        ball_y = self.vision_data['ball_y']
        robot_x = self.vision_data['robot_x']
        robot_y = self.vision_data['robot_y']
        robot_theta = self.vision_data['robot_theta']
        
        self.get_logger().info('📊 Approaching ball:')
        self.get_logger().info(f'    Robot: ({robot_x:.1f}, {robot_y:.1f}) cm, θ={math.degrees(robot_theta):.1f}°')
        self.get_logger().info(f'    Ball: ({ball_x:.1f}, {ball_y:.1f}) cm')
        
        # Calculate angle to ball
        dx = ball_x - robot_x
        dy = ball_y - robot_y
        angle_to_ball = math.atan2(dy, dx)
        angle_to_rotate = self.normalize_angle(angle_to_ball - robot_theta)
        
        # Calculate distance to drive
        distance_to_ball = math.sqrt(dx**2 + dy**2)
        self.distance_to_drive_m = (distance_to_ball - self.APPROACH_DISTANCE_CM) / 100.0
        
        self.get_logger().info(f'    Rotating: {math.degrees(angle_to_rotate):.1f}°')
        self.get_logger().info(f'    Will drive: {self.distance_to_drive_m:.2f} m')
        
        if abs(angle_to_rotate) < self.ANGLE_TOLERANCE_RAD:
            self.get_logger().info('✓ Already facing ball')
            self.state = 'DRIVING_TO_BALL'
            self.drive_to_ball()
            return
        
        goal_msg = RotateAngle.Goal()
        goal_msg.angle = angle_to_rotate
        goal_msg.max_rotation_speed = 0.5
        
        send_goal_future = self.rotate_client.send_goal_async(goal_msg)
        send_goal_future.add_done_callback(self.rotate_to_ball_response)
    
    def rotate_to_ball_response(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().error('❌ Rotation REJECTED!')
            self.state = 'WAITING_OUTSIDE'
            return
        
        result_future = goal_handle.get_result_async()
        result_future.add_done_callback(self.rotate_to_ball_complete)
    
    def rotate_to_ball_complete(self, future):
        self.get_logger().info('✓ Rotation complete!')
        self.state = 'DRIVING_TO_BALL'
        self.drive_to_ball()
    
    def drive_to_ball(self):
        """Drive to approach position near ball"""
        if self.distance_to_drive_m < 0:
            self.get_logger().warn('⚠️ Already at ball position')
            self.verify_ball_approach()
            return
        
        goal_msg = DriveDistance.Goal()
        goal_msg.distance = self.distance_to_drive_m
        goal_msg.max_translation_speed = 0.3
        
        self.get_logger().info(f'🚀 Driving to ball...')
        send_goal_future = self.drive_client.send_goal_async(goal_msg)
        send_goal_future.add_done_callback(self.drive_to_ball_response)
    
    def drive_to_ball_response(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().error('❌ Drive REJECTED!')
            self.state = 'WAITING_OUTSIDE'
            return
        
        result_future = goal_handle.get_result_async()
        result_future.add_done_callback(self.drive_to_ball_complete)
    
    def drive_to_ball_complete(self, future):
        self.get_logger().info('✓ Reached ball position!')
        time.sleep(1.0)
        self.verify_ball_approach()
    
    def verify_ball_approach(self):
        """Verify robot is 30cm from ball and facing it"""
        if not self.vision_data or not self.vision_data.get('robot_found') or not self.vision_data.get('ball_found'):
            self.get_logger().warn('⚠️ Cannot verify: Missing vision data')
            self.retry_approach()
            return
        
        ball_x = self.vision_data['ball_x']
        ball_y = self.vision_data['ball_y']
        robot_x = self.vision_data['robot_x']
        robot_y = self.vision_data['robot_y']
        robot_theta = self.vision_data['robot_theta']
        
        dx = ball_x - robot_x
        dy = ball_y - robot_y
        distance = math.sqrt(dx**2 + dy**2)
        
        angle_to_ball = math.atan2(dy, dx)
        angle_diff = abs(self.normalize_angle(angle_to_ball - robot_theta))
        
        self.get_logger().info('🔍 Position Verification:')
        self.get_logger().info(f'    Distance: {distance:.1f} cm (target: {self.APPROACH_DISTANCE_CM:.1f} cm)')
        self.get_logger().info(f'    Angle diff: {math.degrees(angle_diff):.1f}°')
        
        distance_ok = abs(distance - self.APPROACH_DISTANCE_CM) < self.POSITION_TOLERANCE_CM
        orientation_ok = angle_diff < self.ANGLE_TOLERANCE_RAD
        
        if distance_ok and orientation_ok:
            self.get_logger().info('✓ Position verified! Clasping ball...')
            self.state = 'CLASPING_BALL'
            self.retry_count = 0
            self.clasp_ball()
        else:
            self.retry_approach()
    
    def retry_approach(self):
        """Retry approaching ball"""
        self.retry_count += 1
        if self.retry_count < self.MAX_RETRIES:
            self.get_logger().info(f'⚠️ Retry {self.retry_count}/{self.MAX_RETRIES}')
            self.state = 'ROTATING_TO_BALL'
            time.sleep(1.0)
            self.rotate_to_ball()
        else:
            self.get_logger().error('❌ Max retries reached!')
            self.state = 'WAITING_OUTSIDE'
            self.retry_count = 0
    
    def clasp_ball(self):
        """Close clasps around ball"""
        self.get_logger().info('🤲 Closing clasps...')
        
        # Send clasp command to ESP32
        self.publish_servo_command(b'close')
        
        # Wait for clasps to close
        self.get_logger().info(f'Waiting {self.CLASP_WAIT_TIME} seconds for clasp action...')
        time.sleep(self.CLASP_WAIT_TIME)
        
        # Verify ball is clasped
        self.verify_ball_clasped()
    
    def verify_ball_clasped(self):
        """Verify ball is 30cm directly in front after clasping"""
        if not self.vision_data or not self.vision_data.get('robot_found') or not self.vision_data.get('ball_found'):
            self.get_logger().warn('⚠️ Cannot verify clasp: Missing vision data')
            self.retry_clasp()
            return
        
        ball_x = self.vision_data['ball_x']
        ball_y = self.vision_data['ball_y']
        robot_x = self.vision_data['robot_x']
        robot_y = self.vision_data['robot_y']
        robot_theta = self.vision_data['robot_theta']
        
        # Transform ball to robot frame
        dx = ball_x - robot_x
        dy = ball_y - robot_y
        dx_robot = dx * math.cos(-robot_theta) - dy * math.sin(-robot_theta)
        dy_robot = dx * math.sin(-robot_theta) + dy * math.cos(-robot_theta)
        
        distance_front = dx_robot
        distance_side = abs(dy_robot)
        
        self.get_logger().info('🔍 Clasp Verification:')
        self.get_logger().info(f'    Distance in front: {distance_front:.1f} cm')
        self.get_logger().info(f'    Side offset: {distance_side:.1f} cm')
        
        is_correct = (abs(distance_front - self.APPROACH_DISTANCE_CM) < self.POSITION_TOLERANCE_CM 
                      and distance_side < self.POSITION_TOLERANCE_CM)
        
        if is_correct:
            self.get_logger().info('✓ Ball clasped correctly!')
            self.state = 'NAVIGATING_TO_GOAL'
            self.retry_count = 0
            self.navigate_to_shooting_position()
        else:
            self.retry_clasp()
    
    def retry_clasp(self):
        """Retry clasping ball"""
        self.retry_count += 1
        if self.retry_count < self.MAX_RETRIES:
            self.get_logger().info(f'⚠️ Clasp retry {self.retry_count}/{self.MAX_RETRIES}')
            # Open clasps and try again
            self.publish_servo_command(b'open')
            time.sleep(self.CLASP_WAIT_TIME)
            self.state = 'ROTATING_TO_BALL'
            self.rotate_to_ball()
        else:
            self.get_logger().error('❌ Max clasp retries reached!')
            self.state = 'WAITING_OUTSIDE'
            self.retry_count = 0
    
    def navigate_to_shooting_position(self):
        """Navigate to shooting position with ball"""
        self.get_logger().info('=' * 60)
        self.get_logger().info('🎯 NAVIGATING TO SHOOTING POSITION')
        self.get_logger().info('=' * 60)
        
        # First rotate to FACE the shooting position, then drive there
        self.state = 'ROTATING_TO_SHOOT_LOCATION'
        self.rotate_to_face_shooting_position()
    
    def rotate_to_face_shooting_position(self):
        """Rotate to face the shooting position (not the final angle yet)"""
        if not self.vision_data or not self.vision_data.get('robot_found'):
            self.get_logger().error('No robot position!')
            return
        
        robot_x = self.vision_data['robot_x']
        robot_y = self.vision_data['robot_y']
        robot_theta = self.vision_data['robot_theta']
        
        target_x = self.SHOOTING_POSITION_CM['x']
        target_y = self.SHOOTING_POSITION_CM['y']
        
        # Calculate angle to shooting position
        dx = target_x - robot_x
        dy = target_y - robot_y
        angle_to_target = math.atan2(dy, dx)
        angle_to_rotate = self.normalize_angle(angle_to_target - robot_theta)
        
        # Calculate distance for next step
        self.distance_to_shoot_pos_m = math.sqrt(dx**2 + dy**2) / 100.0
        
        self.get_logger().info(f'📊 Navigation to shooting position:')
        self.get_logger().info(f'    Current: ({robot_x:.1f}, {robot_y:.1f}) cm, θ={math.degrees(robot_theta):.1f}°')
        self.get_logger().info(f'    Target: ({target_x:.1f}, {target_y:.1f}) cm')
        self.get_logger().info(f'    Rotating: {math.degrees(angle_to_rotate):.1f}° to face target')
        self.get_logger().info(f'    Will drive: {self.distance_to_shoot_pos_m:.2f} m')
        
        if abs(angle_to_rotate) < self.ANGLE_TOLERANCE_RAD:
            self.get_logger().info('✓ Already facing shooting position')
            self.state = 'DRIVING_TO_SHOOT_POS'
            self.drive_to_shooting_position()
            return
        
        goal_msg = RotateAngle.Goal()
        goal_msg.angle = angle_to_rotate
        goal_msg.max_rotation_speed = 0.3  # Slower with ball
        
        send_goal_future = self.rotate_client.send_goal_async(goal_msg)
        send_goal_future.add_done_callback(self.rotate_to_face_shoot_response)
    
    def rotate_to_face_shoot_response(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().error('❌ Rotation REJECTED!')
            return
        
        result_future = goal_handle.get_result_async()
        result_future.add_done_callback(self.rotate_to_face_shoot_complete)
    
    def rotate_to_face_shoot_complete(self, future):
        self.get_logger().info('✓ Rotation complete! Now driving to position...')
        self.state = 'DRIVING_TO_SHOOT_POS'
        self.drive_to_shooting_position()
    
    def drive_to_shooting_position(self):
        """Drive to shooting position (already facing it)"""
        self.get_logger().info(f'🚀 Driving to shooting position: {self.distance_to_shoot_pos_m:.2f} m')
        
        goal_msg = DriveDistance.Goal()
        goal_msg.distance = self.distance_to_shoot_pos_m
        goal_msg.max_translation_speed = 0.2  # Slow with ball
        
        send_goal_future = self.drive_client.send_goal_async(goal_msg)
        send_goal_future.add_done_callback(self.drive_to_shoot_response)
    
    def drive_to_shoot_response(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().error('❌ Drive REJECTED!')
            return
        
        result_future = goal_handle.get_result_async()
        result_future.add_done_callback(self.drive_to_shoot_complete)
    
    def drive_to_shoot_complete(self, future):
        self.get_logger().info('✓ Reached shooting position!')
        time.sleep(1.0)
        
        # Now rotate to final shooting angle
        self.state = 'ROTATING_TO_SHOOT_ANGLE'
        self.rotate_to_shooting_angle()
    
    def rotate_to_shooting_angle(self):
        """Rotate to final shooting angle (theta = 0)"""
        if not self.vision_data or not self.vision_data.get('robot_found'):
            self.get_logger().error('No robot position!')
            return
        
        robot_theta = self.vision_data['robot_theta']
        target_theta = self.SHOOTING_POSITION_CM['theta']
        angle_to_rotate = self.normalize_angle(target_theta - robot_theta)
        
        self.get_logger().info(f'🔄 Rotating to final shooting angle: {math.degrees(angle_to_rotate):.1f}°')
        self.get_logger().info(f'    Target angle: {math.degrees(target_theta):.1f}°')
        
        if abs(angle_to_rotate) < self.ANGLE_TOLERANCE_RAD:
            self.get_logger().info('✓ Already at shooting angle')
            self.verify_shooting_position()
            return
        
        goal_msg = RotateAngle.Goal()
        goal_msg.angle = angle_to_rotate
        goal_msg.max_rotation_speed = 0.3  # Slower with ball
        
        send_goal_future = self.rotate_client.send_goal_async(goal_msg)
        send_goal_future.add_done_callback(self.rotate_to_shoot_angle_response)
    
    def rotate_to_shoot_angle_response(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().error('❌ Final rotation REJECTED!')
            self.verify_shooting_position()  # Proceed anyway
            return
        
        result_future = goal_handle.get_result_async()
        result_future.add_done_callback(self.rotate_to_shoot_angle_complete)
    
    def rotate_to_shoot_angle_complete(self, future):
        self.get_logger().info('✓ Final rotation complete!')
        time.sleep(0.5)
        self.verify_shooting_position()
    
    def verify_shooting_position(self):
        """Verify at shooting position"""
        if not self.vision_data or not self.vision_data.get('robot_found'):
            self.get_logger().warn('⚠️ Cannot verify shooting position')
            self.prepare_to_shoot()
            return
        
        robot_x = self.vision_data['robot_x']
        robot_y = self.vision_data['robot_y']
        robot_theta = self.vision_data['robot_theta']
        
        target_x = self.SHOOTING_POSITION_CM['x']
        target_y = self.SHOOTING_POSITION_CM['y']
        target_theta = self.SHOOTING_POSITION_CM['theta']
        
        dx = target_x - robot_x
        dy = target_y - robot_y
        distance = math.sqrt(dx**2 + dy**2)
        angle_diff = abs(self.normalize_angle(target_theta - robot_theta))
        
        self.get_logger().info('🔍 Shooting Position Verification:')
        self.get_logger().info(f'    Distance error: {distance:.1f} cm')
        self.get_logger().info(f'    Angle error: {math.degrees(angle_diff):.1f}°')
        
        position_ok = distance < self.POSITION_TOLERANCE_CM
        angle_ok = angle_diff < self.ANGLE_TOLERANCE_RAD
        
        if position_ok and angle_ok:
            self.get_logger().info('✓ Shooting position verified!')
            self.prepare_to_shoot()
        else:
            self.get_logger().warn('⚠️ Position not perfect, but proceeding...')
            self.prepare_to_shoot()
    
    def prepare_to_shoot(self):
        """Open clasps and verify ball position before shooting"""
        self.get_logger().info('🤲 Opening clasps...')
        self.state = 'PREPARING_SHOT'
        
        # Open clasps
        #self.publish_servo_command(b'open')
        #time.sleep(self.CLASP_WAIT_TIME)
        
        # Verify ball is still in position
        if not self.vision_data or not self.vision_data.get('ball_found'):
            self.get_logger().warn('⚠️ Ball not detected after opening clasps!')
            self.retry_count += 1
            if self.retry_count < self.MAX_RETRIES:
                self.get_logger().info('Trying to reclasp...')
                self.clasp_ball()
                return
        
        # Check ball is in front
        ball_x = self.vision_data['ball_x']
        ball_y = self.vision_data['ball_y']
        robot_x = self.vision_data['robot_x']
        robot_y = self.vision_data['robot_y']
        robot_theta = self.vision_data['robot_theta']
        
        dx = ball_x - robot_x
        dy = ball_y - robot_y
        dx_robot = dx * math.cos(-robot_theta) - dy * math.sin(-robot_theta)
        
        if dx_robot > 0 and dx_robot < 40:  # Ball within 40cm in front
            self.get_logger().info('✓ Ball in shooting position!')
            self.shoot_ball()
        else:
            self.get_logger().warn('⚠️ Ball not in correct position!')
            self.retry_count += 1
            if self.retry_count < self.MAX_RETRIES:
                self.clasp_ball()
            else:
                self.shoot_ball()  # Try shooting anyway
    
    def shoot_ball(self):
        """Execute shooting sequence"""
        self.get_logger().info('')
        self.get_logger().info('=' * 60)
        self.get_logger().info('⚽ SHOOTING BALL!')
        self.get_logger().info('=' * 60)
        
        self.state = 'SHOOTING'
        
        # Send shoot command to ESP32
        self.publish_servo_command(b'shoot')
        
        self.get_logger().info('Shot executed! Waiting for goal result...')
        time.sleep(2.0)  # Wait for shot to complete
        
        # Wait for goal status
        self.state = 'WAITING_FOR_RESULT'
        self.wait_for_goal_result()
    
    def wait_for_goal_result(self):
        """Wait and check goal status"""
        timeout = 5.0
        start_time = time.time()
        
        self.get_logger().info('Checking goal camera...')
        
        while time.time() - start_time < timeout:
            if self.goal_status == 'GOAL':
                self.get_logger().info('')
                self.get_logger().info('=' * 60)
                self.get_logger().info('🎉 GOAL SCORED! CELEBRATING!')
                self.get_logger().info('=' * 60)
                self.celebrate()
                return
            elif self.goal_status == 'MISS':
                self.get_logger().info('😞 Shot missed. Returning to start...')
                self.reset_to_start()
                return
            time.sleep(0.5)
        
        self.get_logger().info('⚠️ No goal result detected. Assuming miss...')
        self.reset_to_start()
    
    def celebrate(self):
        """Celebrate by spinning in place"""
        self.state = 'CELEBRATING'
        self.get_logger().info(f'🎊 Spinning for {self.CELEBRATION_TIME} seconds!')
        
        # Calculate spins to fill celebration time
        spins_per_second = 1.0  # One full rotation per second
        num_spins = int(self.CELEBRATION_TIME * spins_per_second)
        
        self.celebration_spin_count = 0
        self.celebration_total_spins = num_spins
        self.execute_celebration_spin()
    
    def execute_celebration_spin(self):
        """Execute one celebration spin"""
        if self.celebration_spin_count >= self.celebration_total_spins:
            self.get_logger().info('✓ Celebration complete!')
            self.reset_to_start()
            return
        
        self.celebration_spin_count += 1
        self.get_logger().info(f'🎉 Spin {self.celebration_spin_count}/{self.celebration_total_spins}')
        
        goal_msg = RotateAngle.Goal()
        goal_msg.angle = 2 * math.pi  # Full rotation
        goal_msg.max_rotation_speed = 1.0  # Fast celebration
        
        send_goal_future = self.rotate_client.send_goal_async(goal_msg)
        send_goal_future.add_done_callback(self.celebration_spin_response)
    
    def celebration_spin_response(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().warn('Celebration spin rejected, continuing...')
            self.execute_celebration_spin()
            return
        
        result_future = goal_handle.get_result_async()
        result_future.add_done_callback(self.celebration_spin_complete)
    
    def celebration_spin_complete(self, future):
        """After one spin, continue celebrating or finish"""
        self.execute_celebration_spin()
    
    def reset_to_start(self):
        """Reset robot to waiting state"""
        self.get_logger().info('')
        self.get_logger().info('=' * 60)
        self.get_logger().info('🔄 RESETTING TO START POSITION')
        self.get_logger().info('=' * 60)
        
        # Reset all state variables
        self.state = 'WAITING_OUTSIDE'
        self.retry_count = 0
        self.ball_last_position = None
        self.ball_stationary_start = None
        self.goal_status = 'WAITING'
        
        self.get_logger().info('Ready for next ball...')
    
    def normalize_angle(self, angle_rad):
        """Normalize angle to [-pi, pi] radians"""
        while angle_rad > math.pi:
            angle_rad -= 2 * math.pi
        while angle_rad < -math.pi:
            angle_rad += 2 * math.pi
        return angle_rad


def main(args=None):
    rclpy.init(args=args)
    node = AutonomousGoalScorer()
    
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        node.get_logger().info('')
        node.get_logger().info('=' * 60)
        node.get_logger().info('🛑 EMERGENCY STOP - User pressed Ctrl+C')
        node.get_logger().info('=' * 60)
        pass
    finally:
        node.mqtt_client.loop_stop()
        node.mqtt_client.disconnect()
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()